# Info

This notebook is used for the QC, alignment, and variant calling of both
1) Whole-genome-sequencing data<br>
2) Sequence-capture data<br><br>
against the *P. polionotus* reference genome

# Libraries & setup

In [34]:
## Set up some basics
import sys,os,re,glob,shutil,pickle,subprocess
import pandas as pd
import numpy as np
import time
%run '~/jupyter/py3_functions.py'

os.chdir('/n/hoekstra_lab_tier1/Users/brock/polionotus/')
picard_pref = 'module load centos6/0.0.1-fasrc01; module load java\n\njava -XX:ParallelGCThreads=1 -Djava.io.tmpdir=`pwd`/tmp -Dsamjdk.buffer_size=131072 -Dsamjdk.compression_level=1 -XX:GCTimeLimit=50 -XX:GCHeapFreeLimit=10'


## Chromosomes
Ppol_genome = '/n/hoekstra_lab_tier1/Lab/assemblies/Ppol_1.3.3_chr_JML/Ppol_1.3.3.chromosomes-unplaced_scaffolds.fasta'
Ppol_sizes = {}
with open('Ppol_sizes.txt','r') as data:
    for line in data:
        line = line.strip()
        chrom,size = line.split('\t')
        Ppol_sizes[chrom] = size
data.close()

## Populations
pops = ['PO','LO']
seqcapture_pops = ['ABM','CBM','PKBM','POALB','POPOL','POSUB','SABM','SEBM','SRIBM']

## Maps to populations
sample_pop_map = {}
with open('poplists/wgs_seq_pops_alternate.txt','r') as data:
    for line in data:
        line = line.strip('\n')
        sample, pop = line.split('\t')
        sample_pop_map[sample] = pop
data.close()
        
## Maps to ubams
seqcapture_map = {}
with open('poplists/seqcapture_ubam_map.txt','r') as data:
    for line in data:
        line = line.strip('\n')
        sample, ubam, SM = line.split('\t')
        seqcapture_map[sample] = [ubam,SM]
data.close()

wgs_map = {}
with open('poplists/wgs_ubam_map.txt','r') as data:
    for line in data:
        line = line.strip('\n')
        sample, ubam, SM = line.split('\t')
        wgs_map[sample] = [ubam,SM]
data.close()

In [39]:
!mkdir -p SRA_upload
with open('poplists/all_ubams.txt','r') as infile:
    for line in infile:
        line = line.strip('\n')
        sample, ubam, dummy = line.split('\t')
        cmd = ('''{picard_pref} -Xmx10g -jar /n/home11/twooldridge/Software/picard/picard.jar SamToFastq '''
               '''I={ubam} FASTQ=SRA_upload/{sample}_interleaved.fq.gz ''' 
               '''INTERLEAVE=TRUE ''').format(sample = sample, ubam = ubam, picard_pref = picard_pref)
        slurm = make_slurm(run = True, id = '%s.2fq' % sample, cmd_string = cmd,  mem = '12000', time = '24:00:00')
infile.close()

Submitted batch job 9741820
Submitted batch job 9741821
Submitted batch job 9741822
Submitted batch job 9741823
Submitted batch job 9741824
Submitted batch job 9741825
Submitted batch job 9741826
Submitted batch job 9741827
Submitted batch job 9741828
Submitted batch job 9741829
Submitted batch job 9741830
Submitted batch job 9741831
Submitted batch job 9741832
Submitted batch job 9741833
Submitted batch job 9741834
Submitted batch job 9741835
Submitted batch job 9741836
Submitted batch job 9741837
Submitted batch job 9741838
Submitted batch job 9741839
Submitted batch job 9741840
Submitted batch job 9741841
Submitted batch job 9741842
Submitted batch job 9741843
Submitted batch job 9741844
Submitted batch job 9741845
Submitted batch job 9741846
Submitted batch job 9741847
Submitted batch job 9741848
Submitted batch job 9741849
Submitted batch job 9741850
Submitted batch job 9741851
Submitted batch job 9741852
Submitted batch job 9741853
Submitted batch job 9741854
Submitted batch job 

Submitted batch job 9742140
Submitted batch job 9742141
Submitted batch job 9742142
Submitted batch job 9742143


# Alignment

## For WGS data
Provide more memory, longer time

In [14]:
!mkdir -p bams
os.chdir('/n/hoekstra_lab_tier1/Users/brock/polionotus/bams/')
for sample, [ubam,SM] in wgs_map.items():
    cmd = ('''bash /n/home11/twooldridge/scripts/bwa_aln_best_practices.sh -n 4 -m 20000 -b {ubam} -r {Ppol_genome} -i {sample_id} ''').format(ubam=ubam,Ppol_genome=Ppol_genome,sample_id=sample)
    # Submit
    #!$cmd

## For seqcapture data

This little bash script aligns the sequence capture fastq files to the P. polionotus reference genome

In [15]:
os.chdir('/n/hoekstra_lab_tier1/Users/brock/polionotus/bams/')
for sample, [ubam,SM] in seqcapture_map.items():
    cmd = ('''bash /n/home11/twooldridge/scripts/bwa_aln_best_practices.sh -n 4 -m 10000 -b {ubam} -r {Ppol_genome} -i {sample_id} ''').format(ubam=ubam,Ppol_genome=Ppol_genome,sample_id=sample)
    # Submit
    #!$cmd

Correct sample tags (SM) to have one sample name per bam

In [4]:
os.chdir('/n/hoekstra_lab_tier1/Users/brock/polionotus/bams/')
for sample, [ubam, SM] in seqcapture_map.items():
    bam = glob.glob('%s.mdup2500.bam' % (sample))
    if len(bam) != 1:
        print(sample)
        continue
    bam = bam[0]
    newSM = sample
    # It took FOREVER to find a sed string that would work
    sed_string = "sed -r 's/[^\\t ]*SM[^\\t ]*/SM:{newSM}/ig'".format(newSM=newSM)
    outbam = re.sub("mdup2500.bam","mdup2500.SM.bam",bam)
    cmd=('''module load samtools\n'''
        '''samtools view -H {bam} | {sed_string} | samtools reheader - {bam} > {outbam}''').format(bam=bam,sample=sample,sed_string=sed_string,outbam=outbam)
    #slurm=make_slurm(echo=True,id='%s.repairSM' % os.path.basename(bam),cmd_string=cmd,mem='50000',time='03:00:00')

FASTQC post alignment, dup marking, etc

In [6]:
fastqc_executable = '~/Software/FastQC/fastqc'
fastqc_results = 'QC/fastqc/'
!mkdir -p $fastqc_results

finalbams = glob.glob('bams/*mdup2500.SM.bam')

for bam in finalbams:
    cmd = ('''{fastqc_executable} '''
           '''-o {fastqc_results} '''
           '''{bam}''').format(fastqc_executable=fastqc_executable,
                               fastqc_results=fastqc_results,
                               bam=bam)
    
    # These jobs run pretty  quickly. So you can either submit very short slurm jobs (see below), or just run these 
    # in the loop (no parallelization) and go work on something else for a while
    # slurm option:
    slurm = make_slurm(run=True,id=(os.path.basename(bam) + '.fastqc'),cmd_string=cmd,time='24:15:00',mem='3000',cmd_path='./QC/cmds',log_path = './QC/logs')

Combine results

In [6]:
cmd='multiqc -o QC/bam_QC_summary QC/fastqc/*dup* '
slurm = make_slurm(run=True,id=('multiqc'),cmd_string=cmd,mem='5000',time='00:45:00',cmd_path='QC/cmds/',log_path='QC/logs/')

# Haplotype caller

## Determine sex
Using coverage to determine sex of each sample. The code chunk below calculates coverage for chr23 and chrX, and compares

In [22]:
!mkdir -p misc
combined_map = {}
with open('sample_mapped_bam.txt','r') as infile:
    for line in infile:
        line = line.strip('\n')
        sample, bam = line.split('\t')
        combined_map[sample] = bam
infile.close()
for sample, bam in combined_map.items():
   region1 = 'chr23'
   region2 = 'chrX'
   ## Awk command
   awk_cmd = 'awk \'{sum+=$3} END {print sum/NR}\''
   outfile = '%s.quickdepth.txt' % (sample)
   full_cmd = ('''module load samtools\n'''
               '''d1=$(samtools depth -a -r {region1} {bam} | {awk_cmd})\n'''
               '''d2=$(samtools depth -a -r {region2} {bam} | {awk_cmd})\n'''
               '''echo -e "{sample} chr23 $d1\\n{sample} chrX $d2" > misc/{outfile}''').format(region1=region1,region2=region2,sample=sample,bam  = bam,awk_cmd=awk_cmd,outfile=outfile)
   slurm = make_slurm(run=True,id='%s.quickdepth' % (sample),cmd_string=full_cmd,mem='2000',time='00:10:00')
   

Submitted batch job 9676863
Submitted batch job 9676865
Submitted batch job 9676868
Submitted batch job 9676870
Submitted batch job 9676872
Submitted batch job 9676873
Submitted batch job 9676876
Submitted batch job 9676879
Submitted batch job 9676882
Submitted batch job 9676884
Submitted batch job 9676886
Submitted batch job 9676889
Submitted batch job 9676892
Submitted batch job 9676894
Submitted batch job 9676896
Submitted batch job 9676898
Submitted batch job 9676900
Submitted batch job 9676903
Submitted batch job 9676905
Submitted batch job 9676906
Submitted batch job 9676908
Submitted batch job 9676911
Submitted batch job 9676914
Submitted batch job 9676917
Submitted batch job 9676918
Submitted batch job 9676920
Submitted batch job 9676923
Submitted batch job 9676925
Submitted batch job 9676927
Submitted batch job 9676928
Submitted batch job 9676930
Submitted batch job 9676931
Submitted batch job 9676934
Submitted batch job 9676938
Submitted batch job 9676940
Submitted batch job 

Submitted batch job 9677619
Submitted batch job 9677622
Submitted batch job 9677625
Submitted batch job 9677628


Read in coverage results to determine sample sex. I went ahead and combine the per-sample output on the command line

In [23]:
%%bash
cat misc/*quickdepth* > misc/alldepth.txt

In [25]:
sample_sexes = {}
with open('misc/alldepth.txt','r') as data:
    for line in data:
        line = line.strip()
        sample, chrom, depth = line.split(' ')
        if sample not in sample_sexes.keys():
            sample_sexes[sample] = [depth]
        else:
            sample_sexes[sample].append(depth)
for sample in sample_sexes.keys():
    if float(sample_sexes[sample][1])/float(sample_sexes[sample][0]) < 0.65:
        sample_sexes[sample] = 'M'
    else:
        sample_sexes[sample] = 'F'  

#with open('ID_sexes.txt','w') as data:
#    for ID,sex in sample_sexes.items():
#        ID = re.sub('-2','',ID)
#        data.write('%s\t%s\n' % (ID,sex))
#data.close()

## Submit jobs for WGS data

In [ ]:
!mkdir -p vcfs
os.chdir('/n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs')
for sample in wgs_map.keys():
    sex = sample_sexes[sample]
    !mkdir -p $sample
    bam_files = glob.glob('/n/hoekstra_lab_tier1/Users/brock/polionotus/bams/%s.mdup2500.SM.bam' % sample)
    if len(bam_files) != 1:
        print(sample)
        continue
    bam_string = ''
    for bam in bam_files:
        bam_string = bam_string + '-I %s ' % bam
    for chrom, size in Ppol_sizes.items():
        GVCF_CMD=('''module load centos6/0.0.1-fasrc01; module load java\n\n'''
                  '''java -Xmx3G -XX:ParallelGCThreads=1 -Djava.io.tmpdir=`pwd`/tmp -jar ~/Software/GATK/GenomeAnalysisTK-3.8.jar -T HaplotypeCaller '''
                  '''-R {Ppol_genome} {bam_string}'''
                  '''--heterozygosity 0.005 --genotyping_mode DISCOVERY -ERC GVCF -o {sample}/{chrom}.g.vcf -L {chrom}''').format(Ppol_genome=Ppol_genome,sample=sample,bam_string=bam_string,chrom=chrom)
        if chrom == "chrX" and sex == 'M':
            GVCF_CMD = GVCF_CMD + ' -ploidy 1'
        slurm = make_slurm(echo=True,id=('%s.%s.hapcal' % (sample,chrom)),cmd_string=GVCF_CMD,mem='5000',time='150:00:00',cmd_path = 'cmds',log_path='logs')
        
os.chdir('../')

## Submit jobs for seqcapture data
Submit haplotype caller jobs per chromosome

In [ ]:
!mkdir -p vcfs
os.chdir('/n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs')
for sample in seqcapture_map.keys():
    sex = sample_sexes[sample]
    !mkdir -p $sample
    bam_files = glob.glob('/n/hoekstra_lab_tier1/Users/brock/polionotus/bams/%s.mdup2500.SM.bam' % sample)
    if len(bam_files) != 1:
        print(sample)
        continue
    bam_string = ''
    for bam in bam_files:
        bam_string = bam_string + '-I %s ' % bam
    for chrom, size in Ppol_sizes.items():
        GVCF_CMD=('''module load centos6/0.0.1-fasrc01; module load java\n\n'''
                  '''java -Xmx3G -XX:ParallelGCThreads=1 -Djava.io.tmpdir=`pwd`/tmp -jar ~/Software/GATK/GenomeAnalysisTK-3.8.jar -T HaplotypeCaller '''
                  '''-R {Ppol_genome} {bam_string}'''
                  '''--heterozygosity 0.005 --genotyping_mode DISCOVERY -ERC GVCF -o {sample}/{chrom}.g.vcf -L {chrom}''').format(Ppol_genome=Ppol_genome,sample=sample,bam_string=bam_string,chrom=chrom)
        if chrom == "chrX" and sex == 'M':
            GVCF_CMD = GVCF_CMD + ' -ploidy 1'
        #slurm = make_slurm(run=True,id=('%s.%s.hapcal' % (sample,chrom)),cmd_string=GVCF_CMD,mem='5000',time='02:00:00',cmd_path = 'cmds',log_path='logs')
        
os.chdir('../')

Very quick check to see which things have not completed successfully

In [18]:
os.chdir('/n/hoekstra_lab_tier1/Users/Users/brock/polionotus/vcfs')
for sample in seqcapture_map.keys():
    for chrom in Ppol_sizes.keys():
        if len(glob.glob('%s/%s.g.vcf' % (sample,chrom))) == 0:
            print('%s' % (sample))
            continue
        continue

ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
ABM5
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
SABM24
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD2
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
VSD7
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104
PKBM104


# Joint genotyping

## For WGS data

Set up groups

In [40]:
groups = {'polionotus':wgs_map.keys()}

Run joint genotyping

In [43]:
os.chdir('/n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs/')
!mkdir -p cmds logs

module_cmd = 'module load centos6/0.0.1-fasrc01\nmodule load java\n'
base_cmd=('''java -Xmx40G -XX:ParallelGCThreads=1 -Djava.io.tmpdir=`pwd`/tmp -jar ~/Software/GATK/GenomeAnalysisTK-3.8.jar -T GenotypeGVCFs '''
          '''-R {ref_genome} '''
          '''-newQual -allSites ''').format(ref_genome=Ppol_genome)

allsites = True

# Now loop through the groups and create commands
for grp,samples in groups.items():
    for chrom in Ppol_sizes.keys():
        gvcf_string = ''
        for sample in samples:
            gvcf_string = gvcf_string + ' --variant /n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs/' + sample + '/' + chrom + '.g.vcf'
        gatk_cmd = base_cmd + gvcf_string + ' -o %s.allSites.vcf\n' % chrom 
        zip_cmd = 'bash /n/home11/twooldridge/scripts/zip_index_vcf.sh %s.allSites.vcf\n' % chrom
        full_cmd = module_cmd + gatk_cmd + zip_cmd
        #SLURM = make_slurm(echo=True,id='%s.jointgenotype' % (chrom),cmd_string=full_cmd,mem='50000',time='250:00:00',n=2,p='hoekstra,commons')



## For seqcapture data

In [47]:
!mkdir -p /n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs/seqcapture
os.chdir('/n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs/seqcapture')
!mkdir -p cmds logs

module_cmd = 'module load centos6/0.0.1-fasrc01\nmodule load java\n'
base_cmd=('''java -Xmx40G -XX:ParallelGCThreads=1 -Djava.io.tmpdir=`pwd`/tmp -jar ~/Software/GATK/GenomeAnalysisTK-3.8.jar -T GenotypeGVCFs '''
          '''-R {ref_genome} '''
          '''-newQual --disable_auto_index_creation_and_locking_when_reading_rods''').format(ref_genome=Ppol_genome)

allsites = True

#Now loop through the groups and create commands

for chrom in Ppol_sizes.keys():
    gvcf_string = ''
    for sample in seqcapture_map.keys():
        gvcf_string = gvcf_string + ' --variant /n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs/%s/%s.g.vcf' % (sample,chrom)
    gatk_cmd = base_cmd + gvcf_string + ' -o %s.vcf\n' % chrom 
    zip_cmd = 'bash /n/home11/twooldridge/scripts/zip_index_vcf.sh %s.vcf\n' % chrom
    full_cmd = module_cmd + gatk_cmd + zip_cmd
    #SLURM = make_slurm(echo=True,id='%s.jointgenotype' % (chrom),cmd_string=full_cmd,mem='50000',time='100:00:00',n=2,p='hoekstra,commons')



# Variant filtering

## For WGS data

In [3]:
os.chdir('/n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs/')
for chrom in Pman_sizes.keys():
    cmd = ('''/n/home11/twooldridge/scripts/vcf_filter_pipeline.sh '''
           '''{chrom}.allSites.vcf.gz '''
           '''{Pman_genome} '''
           '''{chrom}''').format(chrom=chrom,Ppol_genome = Ppol_genome)
    #print(cmd)

## For seqcapture data

In [5]:
os.chdir('/n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs/seqcapture/')
for chrom in Ppol_sizes.keys():
    cmd = ('''/n/home11/twooldridge/scripts/vcf_filter_pipeline.sh '''
           '''{chrom}.vcf '''
           '''{Ppol_genome} '''
           '''{chrom}''').format(chrom=chrom,Ppol_genome=Ppol_genome)
    #print(cmd)

Submitted batch job 26914060
Submitted batch job 26914064
Submitted batch job 26914069
Submitted batch job 26914073
Submitted batch job 26914077
Submitted batch job 26914086
Submitted batch job 26914091
Submitted batch job 26914094
Submitted batch job 26914102
Submitted batch job 26914106
Submitted batch job 26914111
Submitted batch job 26914118
Submitted batch job 26914122
Submitted batch job 26914127
Submitted batch job 26914130
Submitted batch job 26914138
Submitted batch job 26914142
Submitted batch job 26914147
Submitted batch job 26914152
Submitted batch job 26914160
Submitted batch job 26914163
Submitted batch job 26914170
Submitted batch job 26914176
Submitted batch job 26914179


# Combine WGS and seqcapture data

Get sites from seqcapture data that are at least 50% genotyped. Anything less I'm probably not interested in

In [63]:
outdir = '/n/hoekstra_lab_tier1/Users/brock/polionotus/popgen_vcfs/'
os.chdir(outdir)
!mkdir -p cmds logs vars
for chrom in Ppol_sizes.keys():
    cmd=('''module load vcftools\n'''
         '''vcftools --gzvcf /n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs/seqcapture/hf.{chrom}.vcf.gz '''
         '''--max-missing 0.5 --kept-sites --out vars/{chrom}''').format(chrom=chrom)
    slurm = make_slurm(echo=True,id='%s.sites' % chrom,cmd_string=cmd,mem='1000',time='01:00:00')

Now grab these same sites from the whole genome data

In [5]:
outdir = '/n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/popgen_vcfs/'
os.chdir(outdir)
!mkdir -p cmds logs vars

for chrom in Ppol_sizes.keys():
    cmd=('''module load bcftools;module load vcftools\n'''
         '''vcftools --gzvcf /n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs/polionotus/{chrom}.allSites.vcf.gz '''
         '''--remove-filtered-all --positions <(tail -n +2 vars/{chrom}.kept.sites) --recode --recode-INFO-all --stdout > '''
         '''vars/WGS.{chrom}.vcf\n'''
         '''/n/home11/twooldridge/scripts/misc/zip_index_vcf.sh vars/WGS.{chrom}.vcf''').format(chrom=chrom)
    slurm = make_slurm(echo=True,id='%s.WGSfilter' % chrom,cmd_string = cmd,mem='5000',time='03:00:00')

Submitted batch job 33456829
Submitted batch job 33456830
Submitted batch job 33456831
Submitted batch job 33456832
Submitted batch job 33456833
Submitted batch job 33456834
Submitted batch job 33456835
Submitted batch job 33456837
Submitted batch job 33456838
Submitted batch job 33456839
Submitted batch job 33456840
Submitted batch job 33456841
Submitted batch job 33456842
Submitted batch job 33456843
Submitted batch job 33456844
Submitted batch job 33456845
Submitted batch job 33456846
Submitted batch job 33456847
Submitted batch job 33456848
Submitted batch job 33456849
Submitted batch job 33456850
Submitted batch job 33456851
Submitted batch job 33456852
Submitted batch job 33456853


And from the NUBITERRAE (outgroup) data

In [6]:
outdir = '/n/hoekstra_lab_tier1/Users/brock/polionotus/popgen_vcfs/'
os.chdir(outdir)
!mkdir -p cmds logs vars

for chrom in Ppol_sizes.keys():
    cmd=('''module load vcftools\n'''
         '''vcftools --gzvcf /n/hoekstra_lab_tier1/Users/brock/polionotus/ancestral/01_NUB/{chrom}.allSites.vcf.gz '''
         '''--remove-filtered-all --positions <(tail -n +2 vars/{chrom}.kept.sites) --recode --recode-INFO-all --stdout > '''
         '''vars/NUB.{chrom}.vcf\n'''
         '''/n/home11/twooldridge/scripts/misc/zip_index_vcf.sh vars/NUB.{chrom}.vcf''').format(chrom=chrom)
    slurm = make_slurm(run=True,id='%s.NUBfilter' % chrom,cmd_string = cmd,mem='5000',time='03:00:00')

Submitted batch job 33456854
Submitted batch job 33456855
Submitted batch job 33456856
Submitted batch job 33456857
Submitted batch job 33456858
Submitted batch job 33456859
Submitted batch job 33456860
Submitted batch job 33456861
Submitted batch job 33456862
Submitted batch job 33456863
Submitted batch job 33456864
Submitted batch job 33456865
Submitted batch job 33456866
Submitted batch job 33456867
Submitted batch job 33456868
Submitted batch job 33456869
Submitted batch job 33456870
Submitted batch job 33456871
Submitted batch job 33456872
Submitted batch job 33456880
Submitted batch job 33456881
Submitted batch job 33456882
Submitted batch job 33456883
Submitted batch job 33456884


Finally, retrieve these sites form the seqcapture data

In [69]:
outdir = '/n/hoekstra_lab_tier1/Users/brock/polionotus/popgen_vcfs/'
os.chdir(outdir)
!mkdir -p cmds logs vars

for chrom in Ppol_sizes.keys():
    cmd=('''module load vcftools\n'''
         '''vcftools --gzvcf /n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs/seqcapture/hf.{chrom}.vcf.gz '''
         '''--remove-filtered-all --positions <(tail -n +2 vars/{chrom}.kept.sites) --recode --recode-INFO-all --stdout > '''
         '''vars/SEQ.{chrom}.vcf\n'''
         '''/n/home11/twooldridge/scripts/misc/zip_index_vcf.sh vars/SEQ.{chrom}.vcf''').format(chrom=chrom)
    slurm = make_slurm(run=True,id='%s.SEQfilter' % chrom,cmd_string = cmd,mem='5000',time='01:00:00')

Now use vcftools merge to combine the data. I'm running this with default params, which means that if no data exists for an individual at a site (as will happen with a SEQCAPTURE individual), then we'll just assume a missing `./.` genotype. 

In [7]:
outdir = '/n/hoekstra_lab_tier1/Users/brock/polionotus/popgen_vcfs/'
os.chdir(outdir)
!mkdir -p cmds logs vars

for chrom in Ppol_sizes.keys():
    cmd=('''module load vcftools\n'''
         '''vcf-merge '''
         '''vars/WGS.{chrom}.vcf.gz '''
         '''vars/NUB.{chrom}.vcf.gz '''
         '''vars/SEQ.{chrom}.vcf.gz > '''
         '''vars/ALL.{chrom}.vcf\n'''
         '''/n/home11/twooldridge/scripts/misc/zip_index_vcf.sh vars/ALL.{chrom}.vcf''').format(chrom=chrom)
    slurm = make_slurm(run=True,id='%s.merge' % chrom,cmd_string = cmd,mem='1000',time='01:00:00')

ERROR! Session/line number was not unique in database. History logging moved to new session 1480
Submitted batch job 33457235
Submitted batch job 33457236
Submitted batch job 33457237
Submitted batch job 33457238
Submitted batch job 33457239
Submitted batch job 33457240
Submitted batch job 33457241
Submitted batch job 33457242
Submitted batch job 33457243
Submitted batch job 33457244
Submitted batch job 33457245
Submitted batch job 33457246
Submitted batch job 33457247
Submitted batch job 33457248
Submitted batch job 33457249
Submitted batch job 33457250
Submitted batch job 33457251
Submitted batch job 33457252
Submitted batch job 33457253
Submitted batch job 33457254
Submitted batch job 33457255
Submitted batch job 33457256
Submitted batch job 33457257
Submitted batch job 33457258


Finally, combine all vcfs

In [10]:
os.chdir('/n/hoekstra_lab_tier1/Users/brock/polionotus/popgen_vcfs/')
vcf_string = ''
for chrom in Ppol_sizes.keys():
    vcf_string = vcf_string + 'vars/ALL.%s.vcf.gz ' % chrom
cmd = ('''module load vcftools\n'''
       '''vcf-concat %s > vars/ALL.vcf\n'''
       '''/n/home11/twooldridge/scripts/zip_index_vcf.sh vars/ALL.vcf''' % vcf_string )
slurm = make_slurm(run=True,id='concat',cmd_string = cmd,mem='5000',time='01:00:00')

## Look at site depth in seqcapture data

First, look at vcfs

In [21]:
outdir = '/n/hoekstra_lab_tier1/Users/brock/polionotus/vcfs/seqcapture'
for chrom in Ppol_sizes.keys():
    cmd = ('''module load bcftools\n'''
           '''bcftools query -f '%CHROM %POS %REF %ALT %AC %AN\\n' '''
           '''/n/hoekstra_lab_tier1/Users/brock/polionotus/popgen_vcfs/vars/ALL.{chrom}.vcf.gz > {outdir}/{chrom}.counts.txt''').format(chrom=chrom,outdir=outdir)
    slurm = make_slurm(run=True,id='%s.query' % chrom,cmd_string=cmd,mem='1000',time='00:05:00')
    

Combine files

In [22]:
%%bash
cd /n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/vcfs/seqcapture/
cat chr*counts.txt > all.counts.txt

Look at site depth from bam files

In [93]:
for sample in seqcapture_map.keys():
    bam = glob.glob('../bams/%s.mdup2500.bam' % sample)
    if len(bam) != 1:
        print(sample)
    bam = bam[0]
    cmd = ('''module load samtools\n'''
           '''samtools depth -a -b ../beds/agouti_mc1r_extended.bed '''
           '''{bam} > ../bams/{sample}.depth.txt''').format(sample=sample,bam=bam)
    slurm = make_slurm(run=True,id='%s.depth' % sample,cmd_string = cmd,mem='10000',time='00:10:00')

Combine files

In [99]:
%%bash
rm ../bams/agouti_mc1r_depth.txt
touch ../bams/agouti_mc1r_depth.txt
for file in ../bams/*.depth.txt;do
    sample=$(basename $file | sed 's|.depth.txt||');
    #echo $file;echo $sample;
    cat <(sed -e "s/$/\t$sample/" $file) >> ../bams/agouti_mc1r_depth.txt 
done

rm: cannot remove ‘../bams/agouti_mc1r_depth.txt’: No such file or directory


In [90]:
for sample in seqcapture_map.keys():
    bam = glob.glob('../bams/%s.mdup2500.bam' % sample)
    if len(bam) != 1:
        print(sample)
    bam = bam[0]
    cmd = ('''mosdepth -b /n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/beds/agouti_mc1r_extended.bed '''
           '''--fast-mode ../bams/{sample} {bam}''').format(sample=sample,bam=bam)
    slurm = make_slurm(echo=True,id='%s.depth' % sample,cmd_string = cmd,mem='10000',time='02:00:00')

#!/bin/bash
#SBATCH -p hoekstra,commons,shared
#SBATCH -t 02:00:00
#SBATCH --mem=10000
#SBATCH -n 1
#SBATCH -e ./logs/VSD142.depth.e
#SBATCH -o ./logs/VSD142.depth.o
#SBATCH -c 1
#SBATCH -N 1
#SBATCH -J VSD142.depth

mosdepth -b /n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/beds/agouti_mc1r_extended.bed --fast-mode ../bams/VSD142 ../bams/VSD142.mdup2500.bam

#!/bin/bash
#SBATCH -p hoekstra,commons,shared
#SBATCH -t 02:00:00
#SBATCH --mem=10000
#SBATCH -n 1
#SBATCH -e ./logs/VSD143.depth.e
#SBATCH -o ./logs/VSD143.depth.o
#SBATCH -c 1
#SBATCH -N 1
#SBATCH -J VSD143.depth

mosdepth -b /n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/beds/agouti_mc1r_extended.bed --fast-mode ../bams/VSD143 ../bams/VSD143.mdup2500.bam

#!/bin/bash
#SBATCH -p hoekstra,commons,shared
#SBATCH -t 02:00:00
#SBATCH --mem=10000
#SBATCH -n 1
#SBATCH -e ./logs/VSD195.depth.e
#SBATCH -o ./logs/VSD195.depth.o
#SBATCH -c 1
#SBATCH -N 1
#SBATCH -J VSD195.depth

mosdepth -b /n/holylfs03/LABS/hoekstra_lab/Us

limit_output extension: Maximum message size of 10000 exceeded with 47232 characters

In [79]:
cmd = ('''module load vcftools\n '''
       '''vcftools --gzvcf vars/ALL.vcf.gz --site-depth --out all''')
slurm = make_slurm(run=True,id='depth',cmd_string = cmd,mem='10000',time='02:00:00')

# Generating data to test effects of coverage 

## Downsample WGS data

In [17]:
os.chdir('/n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/bams/downsample/')
for pop, samples in pop_sample_map.items():
    for sample in samples:
        bam = '/n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/bams/%s.combined.mdup2500.SM.bam' % sample
        cmd = ('''source activate py36\n'''
               '''sambamba view -h -f bam -t 8 -s 0.10 {bam} > {sample}.DS.bam\n'''
               '''sambamba index {sample}.DS.bam''').format(bam=bam,sample=sample)
        slurm = make_slurm(run=True,id='%s.downsample' % sample,cmd_string=cmd,mem='5000',time='06:00:00',n=8)

Use samtools mpileup and bcftools call to get potentailly quicker variant calls

In [8]:
!mkdir -p cmds logs
for chrom in Ppol_sizes.keys():
    sample_string = ''
    for sample in flatten(pop_sample_map.values()):
        sample_string = sample_string + '/n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/bams/downsample/%s.DS.bam ' % sample
    cmd = ('''module load samtools bcftools\n'''
           '''samtools mpileup -r {chrom} -v {sample_string} -f {ref_genome} | /n/home11/twooldridge/Software/bcftools-1.3.1/bcftools call -m -v > vcfs/{chrom}.DS.vcf ''').format(chrom=chrom,sample_string=sample_string,ref_genome = Ppol_genome)
    slurm = make_slurm(run=True,id='%s.DS.call' % chrom,cmd_string=cmd,mem='30000',time='72:00:00')

Grab seqcapture sites from these new files

In [5]:
outdir = '/n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/popgen_vcfs/'
os.chdir(outdir)
!mkdir -p cmds logs vars

for chrom in Ppol_sizes.keys():
    cmd=('''module load bcftools;module load vcftools\n'''
         '''vcftools --gzvcf /n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/vcfs/{chrom}.DS.vcf.gz '''
         '''--remove-filtered-all --positions <(tail -n +2 vars/{chrom}.kept.sites) --recode --recode-INFO-all --stdout > '''
         '''vars/WGS.DS.{chrom}.vcf\n'''
         '''/n/home11/twooldridge/scripts/zip_index_vcf.sh vars/WGS.DS.{chrom}.vcf''').format(chrom=chrom)
    slurm = make_slurm(run=True,id='%s.WGSfilter' % chrom,cmd_string = cmd,mem='5000',time='03:00:00')

Now merge w/ seqcapture vcfs

In [7]:
outdir = '/n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/popgen_vcfs/'
os.chdir(outdir)
!mkdir -p cmds logs vars

for chrom in Ppol_sizes.keys():
    cmd=('''module load vcftools\n'''
         '''vcf-merge '''
         '''vars/WGS.DS.{chrom}.vcf.gz '''
         '''vars/NUB.{chrom}.vcf.gz '''
         '''vars/SEQ.{chrom}.vcf.gz > '''
         '''vars/NUB.DS.{chrom}.vcf\n'''
         '''/n/home11/twooldridge/scripts/zip_index_vcf.sh vars/NUB.DS.{chrom}.vcf''').format(chrom=chrom)
    slurm = make_slurm(run=True,id='%s.merge' % chrom,cmd_string = cmd,mem='1000',time='01:00:00')

Finally, combine all chromosomes into one file

In [9]:
os.chdir('/n/holylfs03/LABS/hoekstra_lab/Users/brock/polionotus/popgen_vcfs/')
vcf_string = ''
for chrom in Ppol_sizes.keys():
    vcf_string = vcf_string + 'vars/NUB.DS.%s.vcf.gz ' % chrom
cmd = ('''module load vcftools\n'''
       '''vcf-concat %s > vars/NUB.DS.vcf\n'''
       '''/n/home11/twooldridge/scripts/zip_index_vcf.sh vars/NUB.DS.vcf''' % vcf_string )
slurm = make_slurm(run=True,id='concat',cmd_string = cmd,mem='5000',time='01:00:00')